# Practice

Use the use an ischemic 10X Visium spatial slide from [Kuppe et al., 2022] `kuppe_heart19.h5ad` dataset to practice:
- calculating ligand-receptor interactions per cell type using rank_aggregate method;
- visualize the top 10 ligand-receptor interactions of each cell type;


`kuppe_heart19.h5ad` is a tissue sample obtained from a patient with myocardial infarction, focusing on the ischemic zone of the heart tissue. 
The slide provides spatially-resolved information about the cellular composition and gene expression patterns within the tissue.

### Environement Setup

In [ ]:
import pandas as pd
import scanpy as sc
import decoupler as dc
import liana as li
from matplotlib import pyplot as plt
# set dpi to 100, to make the notebook smaller
plt.rcParams['figure.dpi'] = 100
import os

datadir = '../../datasets/Hands_on_2_LIANA_MistY/'

### Load and Normalize Data

In [ ]:
adata = sc.read(os.path.join(datadir, 'kuppe_heart19.h5ad'))
adata.layers['counts'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

Now, we assign each spot to the cell type with the highest value in 'compositions' by setting adata.obs['cell_type'].

In [ ]:
adata.obs['cell_type'] = adata.obsm['compositions'].idxmax(axis=1)

In [ ]:
sc.pl.spatial(adata, color=[None, 'cell_type'], size=1.3, palette='Set1') 

### Run rank_aggregate


In [ ]:
li.mt.rank_aggregate(adata, 
                     groupby='cell_type',
                     use_raw =False, # we don't have raw in this dataset
                     resource_name='consensus',
                     expr_prop=0.1,
                     verbose=True)

In [ ]:
adata.obs['cell_type'].unique()

In [ ]:
li.pl.dotplot(adata = adata, 
              colour='magnitude_rank',
              size='specificity_rank',
              inverse_size=True,
              inverse_colour=True,
              source_labels=adata.obs['cell_type'].unique(),
              target_labels=adata.obs['cell_type'].unique(),
              top_n=10, 
              orderby='magnitude_rank',
              orderby_ascending=True,
              figure_size=(25, 6)
             )

However, this pipeline does not account for the spatial context of the spots, and each spot represents a mixture of multiple cell types. To incorporate spatial information, we can use the spatially informed bivariate metrics implemented in LIANA+.